In [ ]:
# Make sure to mount Google Drive first

import os
from google.colab import drive
import yaml
import shutil
from ultralytics import YOLO

In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

In [ ]:

# Set paths
DATASET_PATH = '/content/dataset'  # Local path for dataset
DRIVE_DATASET_PATH = '/content/drive/MyDrive/path/to/your/dataset'  # Update this path
RESULTS_PATH = '/content/drive/MyDrive/yolo_results'  # Where to save results


In [ ]:
# Create necessary directories
#os.makedirs(DATASET_PATH, exist_ok=True)
#os.makedirs(RESULTS_PATH, exist_ok=True)

In [ ]:
# Copy dataset from Drive to local Colab storage for faster access
#!cp -r $DRIVE_DATASET_PATH/* $DATASET_PATH/

In [ ]:
# Read and modify data.yaml
with open(f'{DATASET_PATH}/data.yaml', 'r') as file:
    data = yaml.safe_load(file)

In [ ]:
# Update paths in data.yaml to match Colab structure
data['train'] = f'{DATASET_PATH}/train/images'
data['val'] = f'{DATASET_PATH}/valid/images'
data['test'] = f'{DATASET_PATH}/test/images'

In [ ]:
# Save modified data.yaml
with open(f'{DATASET_PATH}/data.yaml', 'w') as file:
    yaml.dump(data, file)

In [ ]:
# Print dataset information
print("Dataset Information:")
print(f"Number of classes: {len(data['names'])}")
print("Classes:", data['names'])
print("Paths:")
print(f"Train: {data['train']}")
print(f"Validation: {data['val']}")
print(f"Test: {data['test']}")

In [ ]:
# Training configuration
config = {
    'model_size': 'x',  # n, s, m, l, x (smaller to larger)
    'epochs': 100,
    'batch_size': 16,
    'imgsz': 640,
    'patience': 20,
    'device': 0  # Use GPU
}

In [ ]:

# Initialize and train model
model = YOLO(f"yolov8{config['model_size']}.pt")  # Load pretrained model

In [ ]:
# Train the model
results = model.train(
    data=f'{DATASET_PATH}/data.yaml',
    epochs=config['epochs'],
    batch=config['batch_size'],
    imgsz=config['imgsz'],
    patience=config['patience'],
    device=config['device'],
    project=RESULTS_PATH,
    name='yolov8_custom_model',
    pretrained=True,
    optimizer='Adam',  # Using Adam optimizer
    lr0=0.001,  # Initial learning rate
    lrf=0.01,   # Final learning rate fraction
    momentum=0.937,
    weight_decay=0.0005,
    warmup_epochs=3.0,
    warmup_momentum=0.8,
    warmup_bias_lr=0.1,
    box=7.5,    # Box loss gain
    cls=0.5,    # Classification loss gain
    hsv_h=0.015,  # HSV augmentation
    hsv_s=0.7,
    hsv_v=0.4,
    degrees=0.0,  # Rotation augmentation
    translate=0.1,
    scale=0.5,
    shear=0.0,
    perspective=0.0,
    flipud=0.0,
    fliplr=0.5,
    mosaic=1.0,  # Mosaic augmentation
    mixup=0.0,   # Mixup augmentation
    copy_paste=0.0,
    save_period=10  # Save checkpoint every 10 epochs
)

In [ ]:
# Validate the model
results = model.val()

In [ ]:
# Run inference on test set
results = model.predict(source=data['test'], save=True, conf=0.25)

In [ ]:
# Save the final model in PT format
final_model_path = os.path.join(RESULTS_PATH, 'yolov8_custom_model', 'weights', 'best.pt')
backup_model_path = os.path.join(RESULTS_PATH, 'yolov8_custom_model', 'weights', 'last.pt')